In [1]:
# 1. Import packages for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from  IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# 2. Import packages for time analysis
import time
import datetime as dt
from datetime import date
from datetime import timedelta
from datetime import datetime

# 3 Open Teradata Connection. Import packages for time analysis
from pnp_connectors.connection_config_holders import TeradataConfig
from pnp_connectors.quick_create import QuickCreate

# 3.1 create config
td_config = TeradataConfig(
 host="TDP-N0201.deutschepost.dpwn.com",
 secret_user_name=os.getenv("TERADATA_USER"),
 secret_password=os.getenv("TERADATA_PASSWORD")
)

# 3.2 create connection
td_connection = QuickCreate.create_teradata_connection(td_config=td_config)

# 4. Funktion zum Löschen nicht mehr benötigter Tabellen
def drop_table(tab_name):
    try:
        td_connection.execute_sql(f'''DROP TABLE {tab_name}''')
        print(f'''{tab_name} dropped.''')
    except: 
        print(f'''{tab_name} nicht vorhanden''')
         
def create_table(Tabelle, SQL, Index):
    # Falls notwendig löschen der neuen Tabelle, dann Tabelle ohne Daten erstellen und Daten einfügen:
    drop_table(Tabelle)
    td_connection.execute_sql(f'''CREATE MULTISET TABLE {Tabelle} AS ( {SQL}) WITH NO DATA PRIMARY INDEX({Index})''')
    print(Tabelle + ' initialised with PI ' + Index)
    td_connection.execute_sql(f'''INSERT INTO  {Tabelle} {SQL}''')
    test = td_connection.select(f'''select count(1) as anzahl_ds from {Tabelle}''')
    anzahl = str(test['anzahl_ds'].values[0])
    print(f'''{anzahl} DS injected into table {Tabelle}.''')
    print(' ')
    
# 5. test connection
result = td_connection.select("select * FROM DB_NEXTT_DWH.vw_event_zust where sendungs_code = '340434197332847059'")
result.head()

,SENDUNGS_CODE,MODALITAET,EREIGNIS_DATUM,EREIGNIS_ZEITSTEMPEL,LEIT_PLZ,LEIT_STR,LEIT_HNR,LEIT_PRODUKT_CODE,EPOS_KASSEN_NR,FILIAL_DIREKTIONS_NR,FILIAL_NIEDERLASSUNGS_NR,FILIAL_NR,FILIAL_STRASSEN_NAME,FILIAL_PLZ,FILIAL_ORTS_NAME,EPOS_VORGANGSART,EREIGNISTYP,ZIEL_PZ,ZIEL_ZUSTELLBASIS,ZIEL_ZUSTELLBEZIRK,ZIEL_ZUSTELL_ABSCHNITT,SENDUNGSART,EREIGNIS_JAHR,LISTENTYP,LISTEN_NR,LISTEN_POSITION,BENACHRICHTIGUNGS_IDC,NACHNAHME_BETRAG,NACHNAHME_ZAHLUNGSART,ENTGELT_BETRAG,ENTGELT_ZAHLUNGSART,ZOLLEINFUHR_BETRAG,ZAHLSCHEINENTGELT_KENNUNG,WAEHRUNG,STORNOKENNZEICHNUNG,SAMMELBEHAELTER_NR,FOLGETAG,EMPFAENGER,BETRIEBS_NR,ZUSTELLER_NAME,EMPFANGSKENNZEICHNUNG,EMPFAENGER_NAME,AUSWEIS_NR,AUSWEISBEHOERDEN_NAME,AUSWEISKENNZEICHNUNG,AUSWEISGUELTIGKEIT,ERSATZEMPFAENGER_NAME,HSC_ERROR_CODE,PROCESS_ID,DATA_SOURCE_ID,LINE_NO,LOAD_TIMESTAMP,DATA_SOURCE_TYPE_ID,APPL_01_FLAG,APPL_02_FLAG,APPL_03_FLAG,APPL_04_FLAG,APPL_05_FLAG,APPL_06_FLAG,APPL_07_FLAG,APPL_08_FLAG,LP_DATA_SOURCE,HANDSCANNER_VERSION,ZAHLUNGSART,RUECKFUEHRUNGSENTGELT_BETRAG,UNTERZEICHNER_NAME,MANUELLE_EINGABE,SENDUNGSART_2,PRODUKT_TYP,ZUORDNUNGS_ID,TAS_AUFTRAGS_NR,AUFTRAGSKENNZIFFER,UNBAR_BETRAG,BAR_BETRAG,VERSCHLUESSELUNG,CEP_VERS,CEP_MINLESE_VERS,STATIONSKENNUNG,PST_PLZ,PST_NR,UNTERSCHRIFTSART,SST_PST_NEXTT,SENDUNGS_CODE_TYP,ANZ_N_NACHW_PFL_SDG,LFD_NR_TAS_M_SCANN,MEZ_KUNDENNUMMER,PST_KUNDEN_NR,DROPPOINT_NR,DIENST_KZ,JOURNALSATZNR,AUSWEIS_SONDERFALL,NACHNAHME_BETRAG2,ENTGELT_BETRAG2,BELEGPOS_ENDE,LOAD_ID,N_AUFTRAGSNUMMER,POSITIONSNUMMER,ESR_KENNZIFFER,STATIONSKENNUNG_2,LEITINFO_ALT,SHIPMENT_NR,AUSLIEFER_FLAG,ANZAHL_PS,POSITION_NO,LC_GESCANNT,SCANNER_ID,PERSONAL_NR,GEB_DATUM,ALTERSGRENZE,STP_ID,SCAN_ORT,GRUENE_KARTE,FILIAL_OZ,IDP_KORR,VERTRAGS_IDC,SESSION_ID,ZUSTELL_FLAG,TEZU_DATUM,GID,ZAHLUNGSEMPFAENGER,KONTONUMMER,BLZ,VERWENDUNGSZWECK_1,VERWENDUNGSZWECK_2,FREITEXTWERT,FREITEXT,FREITEXT_KORREKT,STAATSANGEHOERIGKEIT,STAATSANGEHOERIGKEIT_KORREKT,AUSWEISARTID,MELDEANSCHRIFT,VERTRAGSABWICKLUNG,FELDVALIDIERUNGFEHLGESCHLAGEN,AST_ID,POSTCARD_NR,EEA_FLAG,EEA_STRASSE,EEA_HNR,BETRIEBS_NR_KN,UNTERZEICHNER_NAME_KN,EVENT_ID,ARCH_ZEITRAUM,ETL1_DATA_SOURCE_ID,ETL1_LOAD_ID,ETL1_LOAD_DTM,ETL1_LINE_NO,IBAN,BIC,SERVICE_KUERZEL,EVENT_PSLZ_ID,ERZEUGUNGSORT_LAENDERCODE,LAGERN_BIS_DT,UNTERSCHRIFT_BILD_ID,UNTERSCHRIFT_BILD_INHALT,UNTERSCHRIFT_BILD_EXISTS_FLAG,AUTHENTIFIZIERUNGS_TYP,UNTERSCHRIFT_BILD_MIME_TYPE,ERZEUGUNGSORT_AGNR,ARCH_FLAG,ESR_ZUSTELL_DATUM,PAKETKASTEN_ID,POSTMAN_POSTNUMBER,POSTMAN_LOGISTIC_PROVIDER,LOGISTIK_NETZ,PPO_VERS,EINLIEFERER_VORNAME,EINLIEFERER_NAME,EINLIEFERER_STRASSE,EINLIEFERER_HNR,EINLIEFERER_PLZ,EINLIEFERER_ORT,EINLIEFERER_LAENDERCODE,EMPFAENGER_VORNAME,EMPFAENGER_STRASSE,EMPFAENGER_HNR,EMPFAENGER_PLZ,EMPFAENGER_ORTSNAME,EMPFAENGER_LAENDERCODE,EVENT_ART_PAKETSHOP,NICHT_ZUSTELLUNGSGRUND,UEBERGABE_VON,UEBERGABE_ZU,EXTERNE_PAKETSHOP_ID,ISBT_PAKETSHOP_NR,PLZ_CODE_PAKETSHOP_NR,PAKETSHOP_LAENGENGRAD,PAKETSHOP_BREITENGRAD,PAKETSHOP_STRASSEN_NAME,PAKETSHOP_HAUSNUMMER,PAKETSHOP_HAUSNUMMER_ZUSATZ,PAKETSHOP_PLZ,PAKETSHOP_ORTS_NAME,PAKETSHOP_LAENDERCODE,PAKETSHOP_INTERNATIONAL_NR,PAKETSHOP_BETRIEBSMODUS,SERVICE_EMPFAENGER_NACHNAME,SERVICE_EMPFAENGER_VORNAME,PAKET_STATUS,FSHDUPLIKAT_FLAG,GUELTIG_FLAG,ODS_EVENT_ID,PRIMARY_CRITERIA,SECONDARY_CRITERIA,CORR_TO_BE_ARCHIVED_ORIGINATOR,CORR_SCRIPT_ID,CORR_INSTRUCTION_ID,CORR_PROTOCOL_VERSION,CORR_ERROR_CODE,CORR_CHECK_CODE,CORR_ORIG_EVENT_ID,CREATION_LOC_ADDRESS_COUNTRY,PROD_LOCATION_CODE_AGNR,FORMAT_ADJUST_SYNTHETIC_EVENT,FORMAT_ADJUST_CORRECTION_TS,FORMAT_ADJUST_REVISER,FORMAT_ADJUST_COMMENT,FORMAT_ADJUST_POST_PROC_ADJUST,FORMAT_ADJUST_ORIG_EVENT_ID,FORMAT_ADJUST_ORIG_EVT_PRED_ID,EVT_MSG_HDR_DS_DSTID,PSLZ_ID,SHIP_ID,VALID_ROUTING_CODE_TYPE_1,VALID_RC_1_COUNTRY_CODE,VALID_RC_1_POSTAL_CODE,VALID_RC_1_STREET_CODE,VALID_RC_1_HOUSE_NUMBER,VALID_RC_1_PRODUCT_CODE_BASIS,VALID_RC1_PRODUCT_CODE_FEATURE,VALID_ROUTING_CODE_TYPE_2,VALID_RC_2_COUNTRY_CODE,VALID_RC_2_POSTAL_CODE,VALID_RC_2_STREET_CODE,VALID_RC_2_HOUSE_NUMBER,VALID_RC_2_PRODUCT_CODE_BASIS,VALID_RC2_PRODUCT_

# Initialisierung Projekt:

In [2]:
now = date.today()
jahr_akt = now.year
monat_akt = now.month 


if monat_akt == 1:
    monat = 12
    jahr = jahr_akt - 1
else:
    monat = monat_akt - 1
    jahr = jahr_akt


datum1 = date(year = jahr, month = monat, day = 1)  # Erster Tag der Analyse
datum2 = date(year = jahr_akt, month = monat_akt, day = 1)  # Erster Tag der Analyse
#datum2 = datum1 + timedelta(days=-2)
datum1_PAN = datum1 + timedelta(days=-10)
datum2_PAN = datum2 + timedelta(days=+10)
print(datum1, datum2, datum1_PAN)


# collecting the data from PZA,PZE
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_1'

SQL = f'''

            Select
                sendungs_code, 
                ERFASSUNGSORT as PZ_NR, 
                avg(gewicht) as gewicht_real,
                min(gewicht_quelle) as gewicht_quelle, 
                avg(volumen_pals) as Volumen,
                avg(laenge) as laenge, 
                avg(breite) as breite, 
                avg(hoehe) as hoehe,
                count(sendungs_code) as anzahl_messungen,
                max(ereignis_zeitstempel) as pza_timestamp, 
                max(ereignis_datum) as pza_datum,
                min(ereignis_typ) as ereignis_typ, 
                max(zeit) as pza_uhrzeit,
                max(ziel_pz * 1) as ziel_pze, 
                max(ZIELKENNUNG_AGNR) as ziel_pze_agnr,
                min(WEIGHT_LEGAL_CODE) as Status_PMES_1, 
                max(WEIGHT_LEGAL_CODE) as Status_PMES_2,
                max(rc_produkt_code) as rc_produkt_code, 
                max(routing_code)  as routing_code
           from (     
                
                    Select 
                        sendungs_code, ERFASSUNGSORT, gewicht, gewicht_quelle, volumen_pals, laenge, breite, hoehe, 
                        ereignis_zeitstempel, ereignis_datum, ereignis_typ, 
                        EXTRACT( HOUR FROM ereignis_zeitstempel)*60+EXTRACT(MINUTE FROM ereignis_zeitstempel) as zeit,
                        cast(ziel_pz as int) as ziel_pz, ZIELKENNUNG_AGNR, WEIGHT_LEGAL_CODE, rc_produkt_code, routing_code
                    from DB_NEXTT.PZA_event
                    where 	ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                            and Gewicht_Quelle in ('W', 'Y')
                            and WEIGHT_LEGAL_CODE = 'LFT'
                            and gewicht > 0  
                            and volumen_pals > 0
                            and round(gewicht/ volumen_pals,0) <= 5
                            and ERFASSUNGSORT not in ('62')


                    union all


                    Select 
                        sendungs_code, ERFASSUNGSORT, gewicht, gewicht_quelle,volumen_pals, laenge, breite, hoehe,  ereignis_zeitstempel, ereignis_datum, ereignis_typ, 
                        EXTRACT( HOUR FROM ereignis_zeitstempel)*60+EXTRACT(MINUTE FROM ereignis_zeitstempel) as zeit,
                        cast(ziel_pz as int) as ziel_pz, ZIELKENNUNG_AGNR, WEIGHT_LEGAL_CODE, rc_produkt_code, routing_code
                    from DB_NEXTT.PZE_event
                    where 	ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                            and Gewicht_Quelle in ('W', 'Y')
                            and WEIGHT_LEGAL_CODE = 'LFT'
                            and gewicht > 0 
                            and volumen_pals > 0
                            and round(gewicht/ volumen_pals,0) <= 5
                            and ERFASSUNGSORT not in ('62')


 
                    
        ) as temp_table
        group by sendungs_code, ERFASSUNGSORT 
 
            '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
    
# collecting the PAN data 
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_2'
SQL = f'''

            select 	shipment_code as sendungs_code, PAN_ID, ekp_no, method_code as Verfahren_Nr, participation as Teilnahme_Nr, 
                    data_source_type_id, load_dtm as event_dt, lp_data_source as Pruefziffer, total_weight as Gewicht_PAN, File_creation_dtm as Erstellugsdatum
            from DB_NEXTT_DWH.vw_PAN_MSG_SHIP 
            where  	load_dtm >= cast('{datum1_PAN}' as date) and load_dtm < cast('{datum2_PAN}'  as date)
                and method_code  in ('01', '06')
                and total_weight is not null and total_weight <= 50
            Qualify (ROW_NUMBER() OVER(PARTITION BY sendungs_code ORDER BY Erstellugsdatum ASC)) = 1
               

        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
    
# collect the parcel shape data from DB_NEXTT_SHIP.VW_PEM_ADD  table
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_3'
SQL = f'''
            Select 		piece_code as sendungs_code,
                        special_type_61 as BAP_feature,
                        tb_load_dtm,
                        case 
                        when BAP_feature.SHAPE is NOT NULL then BAP_feature.SHAPE
                        end as parcel_shape 
             from 		DB_NEXTT_SHIP.VW_PEM_ADD 
            
    
            where 		tb_load_dtm >= cast('{datum1}' as date)  and tb_load_dtm < cast('{datum2}' as date)
                        and parcel_shape IS NOT NULL
                        and sendungs_code in (select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_nat_1 )
            Qualify (ROW_NUMBER() OVER(PARTITION BY sendungs_code ORDER BY tb_load_dtm ASC)) = 1

    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
    
    
#combine all the data from the above tables PAN, PZ and BAP
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_4'
SQL = f'''

            select 	PAN.sendungs_code, PAN.PAN_ID, PAN.ekp_no, PAN.Verfahren_Nr, PAN.Teilnahme_Nr, 
                    PAN.data_source_type_id, PAN.event_dt,
                    PAN.pruefziffer, PAN.Gewicht_PAN, PAN.Erstellugsdatum,
                    pz.Gewicht_REAL, pz.Gewicht_Quelle, pz.Volumen, pz.laenge, pz.breite, pz.hoehe, pz.anzahl_messungen, 
                    pz.pza_timestamp , pz.pza_Datum, pz.ereignis_typ, pz.PZA_Uhrzeit, pz.PZ_NR,
                    case 
                    when bap.parcel_shape is null then 'None'
                    else bap.parcel_shape
                    end as parcel_shape      
            from DBX_DWH_SBX_GB23_PRD.Herkules_nat_2 as PAN
            inner join DBX_DWH_SBX_GB23_PRD.Herkules_nat_1 as PZ
            on 	PAN.sendungs_code = Pz.sendungs_code
            left join DBX_DWH_SBX_GB23_PRD.Herkules_nat_3 as BAP
            on PAN.sendungs_code = BAP.sendungs_code

                    
        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
    
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_nat_1')
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_nat_2')
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_nat_3')
     
# create the target feature- real weight of the parcel is more than 10 and 20    
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_5'
SQL = f'''  
            select  A.*, 
            cast(OREPLACE(cast(A.EKP_No AS VARCHAR(11)) ,'.', '') || A.Verfahren_Nr || A.Teilnahme_Nr AS VARCHAR(14))  as EKP_NO_EXTENTED,
           
            case
                when Gewicht_Real < 20 or Gewicht_PAN >= 20 then 0
                when Gewicht_Real >= 20  then 1
                else 0
           end as Flag_Gewicht_20,
           case
                when Gewicht_Real < 10 or Gewicht_PAN >= 10 then  0
                when Gewicht_Real >= 10  and  Gewicht_Real < 20 then 1
                else 0
               
           end as Flag_Gewicht_10
        from DBX_DWH_SBX_GB23_PRD.Herkules_nat_4 as A    
                    
        '''
Index = 'sendungs_code, EKP_NO_EXTENTED'
create_table(Tabelle, SQL, Index)    
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_nat_4')
    
    
# create a new feature 'volume_grp'
    
# Alte Verteilung mit Overfitting:
#             when volumen < 4 then 'V04'
#             when volumen < 6 then 'V06'
#             when volumen < 9 then 'V09'
#             when volumen < 13 then 'V13'
#             when volumen < 20 then 'V20'
#             when volumen < 30 then 'V30'
#             when volumen < 50 then 'V50'
#             when volumen >= 50 then 'V99'
    
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_6'
SQL = f'''

            select   
                sendungs_code, pruefziffer, ekp_no_extented, ekp_no, Verfahren_Nr, Teilnahme_Nr, 
                event_dt, Gewicht_PAN, Gewicht_REAL, PZ_NR,
                Gewicht_Quelle,Flag_Gewicht_20,Flag_Gewicht_10,
                Volumen, laenge, breite, hoehe, Erstellugsdatum, parcel_shape, 
                case 
                    when volumen < 30 then 'V30'
                    when volumen < 40 then 'V40'
                    when volumen < 50 then 'V50'
                    when volumen < 70 then 'V70'
                    when volumen < 98 then 'V90'
                    when volumen >= 98 then 'V99'
                    else '?'
                end as volumen_grp,
                case
                    when Gewicht_PAN >= 0  and Gewicht_PAN <= 5  then 'G05'
                    when Gewicht_PAN > 5 and Gewicht_PAN <= 10  then 'G10'
                    when Gewicht_PAN > 10 and Gewicht_PAN <= 15 then 'G15'
                    when Gewicht_PAN > 15 and Gewicht_PAN <= 20 then 'G20'
                    when Gewicht_PAN > 20 and Gewicht_PAN <= 30 then 'G30'
                    when Gewicht_PAN > 30 then 'G99'
                    else '?'
                end as Gewicht_PAN_grp

            from DBX_DWH_SBX_GB23_PRD.Herkules_nat_5 as A
            where   hoehe > 0
                    and breite > 0
                    and laenge > 0  
                    and Gewicht_PAN <> 0  and Gewicht_PAN is not null
                    and Gewicht_REAL <> 0 and Gewicht_REAL is not null 
                    and volumen <> 0      and volumen is not null   
                    
        '''
Index = 'sendungs_code, EKP_NO_EXTENTED'
create_table(Tabelle, SQL, Index)    
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_nat_5')           
    

2024-08-01 2024-09-01 2024-07-22
DBX_DWH_SBX_GB23_PRD.Herkules_nat_1 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.Herkules_nat_1 initialised with PI sendungs_code
40210839 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_nat_1.
 
DBX_DWH_SBX_GB23_PRD.Herkules_nat_2 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.Herkules_nat_2 initialised with PI sendungs_code
170204503 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_nat_2.
 
DBX_DWH_SBX_GB23_PRD.Herkules_nat_3 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.Herkules_nat_3 initialised with PI sendungs_code
36930604 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_nat_3.
 
DBX_DWH_SBX_GB23_PRD.Herkules_nat_4 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.Herkules_nat_4 initialised with PI sendungs_code
30273565 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_nat_4.
 
DBX_DWH_SBX_GB23_PRD.Herkules_nat_1 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_nat_2 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_nat_3 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_nat_5 nicht vorhanden
DBX_DWH

In [3]:
#redefining the parcel shape as we are facing error with data type of parcel shape
Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_1' 
SQL = f'''

       SELECT  sendungs_code, Pruefziffer, EKP_NO_EXTENTED, EKP_NO, Verfahren_Nr, Teilnahme_Nr, event_dt, PZ_NR,
               Gewicht_PAN,  gewicht_real,  gewicht_quelle, Flag_Gewicht_20, Flag_Gewicht_10,
               Volumen, laenge, breite, hoehe, Erstellugsdatum, volumen_grp, Gewicht_PAN_grp,

            case 
                when parcel_shape = 'bag' then 'bag' 
                when parcel_shape = 'cuboid' then 'cuboid' 
                when parcel_shape = 'envelope' then 'envelope'
                else 'unknown'
            end as parcel_shape

   FROM DBX_DWH_SBX_GB23_PRD.Herkules_nat_6 
   
    '''
Index = 'sendungs_code, ekp_no_extented, event_dt'
create_table(Tabelle, SQL, Index)

DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_1 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_1 initialised with PI sendungs_code, ekp_no_extented, event_dt
30045172 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_1.
 


In [4]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_2' 
SQL = f'''

    select  distinct sendungs_code,	Pruefziffer,	EKP_NO_EXTENTED,	EKP_NO,	Verfahren_Nr,	Teilnahme_Nr,	event_dt,
               Gewicht_PAN,	gewicht_real,	gewicht_quelle as Gewicht_Quelle_tmp,	Flag_Gewicht_20,
               Flag_Gewicht_10, Volumen,	laenge,	breite,	hoehe,	Erstellugsdatum,	volumen_grp,	parcel_shape, Gewicht_PAN_grp
    from DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_1
   
    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)

DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_2 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_2 initialised with PI sendungs_code
30044564 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_2.
 


In [5]:
#Joining the features from stammdaten table to the validation data table

Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_3' 
SQL = f'''

        select  a.*, 
                b.volumen_grp as D_volumen_grp,
                b.parcel_shape as D_parcel_shape,
                b.Anzahl, b.D_Anzahl,
                b.D_Anz_diff_pan_weights,
                b.D_Anz_diff_Real_weights,
                b.Zeitraum_Tage,
                b.D_Hoehe, b.D_hoehe_min, b.D_hoehe_max,
                b.D_Laenge, b.D_laenge_min,b.D_laenge_max,
                b.D_Breite, b.D_breite_min, b.D_breite_max,
                b.D_Gewicht_real, b.D_Gewicht_real_min,b.D_Gewicht_real_max,
                b.D_Gewicht_pan, b.D_gewicht_pan_min,b.D_gewicht_pan_max,
                b.D_Volumen, b.D_Volumen_min, b.D_Volumen_max,
                b.D_Volumengewicht_real, b.D_Volumengewicht_pan,
                b.D_LBH_Summe, b.D_Volumengewicht_real2, b.D_Volumengewicht_pan2, 
                b.Anzahl_GU_20,b.Anzahl_GU_10,
                b.D_Gewichtsabweichung, 
                b.D_Gewichtsabweichung_min,
                b.D_Gewichtsabweichung_max,
                b.D_Anteil_GU_20, b.D_Anteil_GU_10,
                b.Anzahl_pan20_korrekt,b.Anzahl_pan20,
                b.Anzahl_pan10_korrekt, b.Anzahl_pan10,b.Anzahl_pan_korrekt,b.Anteil_PAN20_korrekt,b.Anteil_PAN10_korrekt,b.Diff_kg_sum
    
       FROM DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_2 AS a
       left join DBX_DWH_SBX_GB23_PRD.GS_Herkules_EKP_STAMMDATEN_NAT_vmadasu AS b
       on a.EKP_NO_EXTENTED = B.EKP_NO_EXTENTED 
          and a.volumen_grp = b.volumen_grp
          and a.parcel_shape = b.parcel_shape



        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)

 

DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_3 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_3 initialised with PI sendungs_code
30044564 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_3.
 


In [6]:
#creating new features only to the training data 

Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_4' 
SQL = f'''
    select  a.*,
        
        case
            when Hoehe is Null or Hoehe = 0 then 0 
            else round(Laenge / Hoehe) 
        end as VH_Laenge_Hoehe,
        case
            when Breite is Null or Breite = 0 then 0 
            else round(Laenge / Breite) 
        end as VH_Laenge_Breite,
        case
            when Breite is Null or Breite = 0 then 0 
            else round(Hoehe / Breite) 
        end as VH_Hoehe_Breite,
        
        case
            when Laenge is Null or Laenge = 0 then 0 
            else round(Volumen / Laenge) 
        end as VH_Volumen_Laenge,
        case
             when Breite is Null or Breite = 0 then 0 
            else round(Volumen / Breite) 
        end as VH_Volumen_Breite,
        
        case
            when Hoehe is Null or Hoehe = 0 then 0
            else round(Volumen / Hoehe) 
        end as VH_Volumen_Hoehe,
      
        case
            when D_Laenge is Null or D_Laenge = 0 then 0 
            else round(Laenge / D_Laenge) 
        end as VH_Laenge_DLaenge,
        case
            when D_Breite is Null or D_Breite = 0 then 0 
            else round(Breite / D_Breite) 
        end as VH_Breite_DBreite,
        case
            when D_Hoehe is Null or D_Hoehe = 0 then 0 
            else round(Hoehe / D_Hoehe) 
        end as VH_Hoehe_DHoehe,
        
        case
            when D_Volumen is Null or D_Volumen = 0 then 0 
            else round(Volumen / D_Volumen) 
        end as VH_Volumen_DVolumen,
        
        case
            when D_Gewicht_real is Null or D_Gewicht_real = 0 then 0 
            else least(10,round(Gewicht_PAN / D_Gewicht_real)) 
        end as VH_Gewicht_DGewicht,
        
        case
            when D_Gewicht_pan is Null or D_Gewicht_pan = 0 then 0 
            else least(10,round(Gewicht_PAN / D_Gewicht_pan)) 
        end as VH_Gewicht_DGewicht_PAN,
        
        greatest(-10,least(15,round(Gewicht_PAN - D_Gewicht_real))) +10 as DGewichtsabweichung1,
        greatest(-10,least(10,round(D_Gewicht_pan - D_Gewicht_real))) +10 as DGewichtsabweichung2,    
        greatest(-10,least(15,round(Gewicht_PAN - D_Gewicht_pan))) +10 as DGewichtsabweichung3,
        greatest(-10,least(15,round(D_Gewicht_real_min - D_Gewicht_real))) as DGewichtsabweichung4,
        greatest(-10,least(15,round(D_Gewicht_real_max - D_Gewicht_real)))  as DGewichtsabweichung5,
        greatest(-10,least(15,round(D_Gewicht_real_max - D_Gewicht_pan)))  as DGewichtsabweichung6,
        
        
        greatest(4,least(20,round((Laenge + Breite + Hoehe)/100))) as LBH_Summe,
        greatest(70,least(150,round((LBH_Summe*1.00 / d_LBH_Summe*1.00) * 100))) as LBH_VH,
        
        least(50,round(Gewicht_pan / Volumen*100)) as Volumengewicht_pan,
        least(15,round(LBH_Summe / Gewicht_pan)) as Volumengewicht_pan2,
        least(15,round(d_LBH_Summe / Gewicht_pan)) as Volumengewicht_pan3,
        
        case when D_Volumengewicht_pan = 0 or D_Volumengewicht_pan is NULl then 0
            else least(10,round(Volumengewicht_pan * 3 / D_Volumengewicht_pan)) end as VH_Volumengewicht_pan,
        case when D_Volumengewicht_pan2 = 0 or D_Volumengewicht_pan2 is NULl then 0
            else least(10,round(Volumengewicht_pan2 * 3 / D_Volumengewicht_pan2)) end as VH_Volumengewicht_pan2
  
        
   FROM DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_3 AS a


   
    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
#display(td.download_table_odbc(sql=f'''select VH_Gewicht_DGewicht, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))
#display(td.download_table_odbc(sql=f'''select VH_Gewicht_DGewicht_PAN, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))
#display(td.download_table_odbc(sql=f'''select DGewichtsabweichung1, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))
#display(td.download_table_odbc(sql=f'''select DGewichtsabweichung2, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))
#display(td.download_table_odbc(sql=f'''select DGewichtsabweichung3, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))

#display(td.download_table_odbc(sql=f'''select LBH_VH, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))
#display(td.download_table_odbc(sql=f'''select Volumengewicht_pan, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))
#display(td.download_table_odbc(sql=f'''select VH_Volumengewicht_pan, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))
#display(td.download_table_odbc(sql=f'''select VH_Volumengewicht_pan2, count(1) as Anzahl from {Tabelle} group by 1 order by 1'''))

DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_4 dropped.
DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_4 initialised with PI sendungs_code
30044564 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_4.
 


In [7]:
# Final validation table with history features and new features 

Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Scoring'
SQL = f'''
   select  * FROM DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_4 

    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)

DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Scoring dropped.
DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Scoring initialised with PI sendungs_code
30044564 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Scoring.
 


In [8]:
display(td_connection.select("select Flag_Gewicht_20, count(1) as anzahl from DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Scoring group by 1 order by 1"))

,Flag_Gewicht_20,anzahl
0,0,29949863
1,1,94701


In [9]:
drop_table('DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_1')
drop_table('DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_2')
drop_table('DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_3')

DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_1 dropped.
DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_2 dropped.
DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Nat_CV_3 dropped.


In [10]:
#Example to check all the  features in the final validation table

result = td_connection.select("select * from DBX_DWH_SBX_GB23_PRD.GS_Herkules_Basisdaten_Scoring where Gewicht_PAN > 50 ")
result.head()

,sendungs_code,Pruefziffer,EKP_NO_EXTENTED,EKP_NO,Verfahren_Nr,Teilnahme_Nr,event_dt,Gewicht_PAN,gewicht_real,Gewicht_Quelle_tmp,Flag_Gewicht_20,Flag_Gewicht_10,Volumen,laenge,breite,hoehe,Erstellugsdatum,volumen_grp,parcel_shape,Gewicht_PAN_grp,D_volumen_grp,D_parcel_shape,Anzahl,D_Anzahl,D_Anz_diff_pan_weights,D_Anz_diff_Real_weights,Zeitraum_Tage,D_Hoehe,D_hoehe_min,D_hoehe_max,D_Laenge,D_laenge_min,D_laenge_max,D_Breite,D_breite_min,D_breite_max,D_Gewicht_real,D_Gewicht_real_min,D_Gewicht_real_max,D_Gewicht_pan,D_gewicht_pan_min,D_gewicht_pan_max,D_Volumen,D_Volumen_min,D_Volumen_max,D_Volumengewicht_real,D_Volumengewicht_pan,D_LBH_Summe,D_Volumengewicht_real2,D_Volumengewicht_pan2,Anzahl_GU_20,Anzahl_GU_10,D_Gewichtsabweichung,D_Gewichtsabweichung_min,D_Gewichtsabweichung_max,D_Anteil_GU_20,D_Anteil_GU_10,Anzahl_pan20_korrekt,Anzahl_pan20,Anzahl_pan10_korrekt,Anzahl_pan10,Anzahl_pan_korrekt,Anteil_PAN20_korrekt,Anteil_PAN10_korrekt,Diff_kg_sum,VH_Laenge_Hoehe,VH_Laenge_Breite,VH_Hoehe_Breite,VH_Volumen_Laenge,VH_Volumen_Breite,VH_Volumen_Hoehe,VH_Laenge_DLaenge,VH_Breite_DBreite,VH_Hoehe_DHoehe,VH_Volumen_DVolumen,VH_Gewicht_DGewicht,VH_Gewicht_DGewicht_PAN,DGewichtsabweichung1,DGewichtsabweichung2,DGewichtsabweichung3,DGewichtsabweichung4,DGewichtsabweichung5,DGewichtsabweichung6,LBH_Summe,LBH_VH,Volumengewicht_pan,Volumengewicht_pan2,Volumengewicht_pan3,VH_Volumengewicht_pan,VH_Volumengewicht_pan2


In [2]:

# Download the data from Teradata:
import time

# It is important to have a schema with write permissions for the teradata interpreter so that the data types can be converted correctly!
startzeit = time.time()

Used_Sandbox = "DBX_DWH_SBX_GB23_PRD"  
Tabelle = "GS_Herkules_Basisdaten_Scoring"
target_schema = 'DBX_DWH_SBX_GB23_PRD'

display(td_connection.select(f'''SELECT * FROM {Used_Sandbox}.{Tabelle} sample 5'''))

GS_validationset =td_connection.select(f'''    SELECT *FROM {Used_Sandbox}.{Tabelle}  sample 500000''')
GS_validationset.head()

endzeit = time.time()-startzeit

print("Der Import hat ", endzeit , "Sekunden gedauert!")



,sendungs_code,Pruefziffer,EKP_NO_EXTENTED,EKP_NO,Verfahren_Nr,Teilnahme_Nr,event_dt,Gewicht_PAN,gewicht_real,Gewicht_Quelle_tmp,Flag_Gewicht_20,Flag_Gewicht_10,Volumen,laenge,breite,hoehe,Erstellugsdatum,volumen_grp,parcel_shape,Gewicht_PAN_grp,D_volumen_grp,D_parcel_shape,Anzahl,D_Anzahl,D_Anz_diff_pan_weights,D_Anz_diff_Real_weights,Zeitraum_Tage,D_Hoehe,D_hoehe_min,D_hoehe_max,D_Laenge,D_laenge_min,D_laenge_max,D_Breite,D_breite_min,D_breite_max,D_Gewicht_real,D_Gewicht_real_min,D_Gewicht_real_max,D_Gewicht_pan,D_gewicht_pan_min,D_gewicht_pan_max,D_Volumen,D_Volumen_min,D_Volumen_max,D_Volumengewicht_real,D_Volumengewicht_pan,D_LBH_Summe,D_Volumengewicht_real2,D_Volumengewicht_pan2,Anzahl_GU_20,Anzahl_GU_10,D_Gewichtsabweichung,D_Gewichtsabweichung_min,D_Gewichtsabweichung_max,D_Anteil_GU_20,D_Anteil_GU_10,Anzahl_pan20_korrekt,Anzahl_pan20,Anzahl_pan10_korrekt,Anzahl_pan10,Anzahl_pan_korrekt,Anteil_PAN20_korrekt,Anteil_PAN10_korrekt,Diff_kg_sum,VH_Laenge_Hoehe,VH_Laenge_Breite,VH_Hoehe_Breite,VH_Volumen_Laenge,VH_Volumen_Breite,VH_Volumen_Hoehe,VH_Laenge_DLaenge,VH_Breite_DBreite,VH_Hoehe_DHoehe,VH_Volumen_DVolumen,VH_Gewicht_DGewicht,VH_Gewicht_DGewicht_PAN,DGewichtsabweichung1,DGewichtsabweichung2,DGewichtsabweichung3,DGewichtsabweichung4,DGewichtsabweichung5,DGewichtsabweichung6,LBH_Summe,LBH_VH,Volumengewicht_pan,Volumengewicht_pan2,Volumengewicht_pan3,VH_Volumengewicht_pan,VH_Volumengewicht_pan2
0,43163313481,7,52010297730101,5201029773,01,01,2024-08-20 12:40:12.780,5.000,0.55,W,0,0,3.0,207.0,154.0,103.0,2024-08-20 12:12:00,V30,cuboid,G05,V30,cuboid,192,1,5,59,100,121.0,96.0,275.0,229.0,155.0,505.0,169.0,96.0,312.0,2.0,0.0,15.0,6.0,5.000,20.000,5.0,2.0,27.0,40.0,120.0,5.0,3.0,1.0,0,0,-4.0,-5.0,-5.0,0.00,0.00,0,0,2,2,3,1.00,100.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,13.0,14.0,9.0,-2.0,13.0,9.0,5.0,100.0,50.0,1.0,1.0,1.0,3.0
1,340434762105956513,00,63662364950101,6366236495,01,01,2024-08-17 18:46:37.630,3.487,3.55,W,0,0,31.0,493.0,397.0,158.0,2024-08-17 18:35:00,V40,bag,G05,V40,bag,613603,342,8332,1174,1793,191.0,60.0,514.0,469.0,163.0,1165.0,388.0,141.0,1176.0,3.0,0.0,30.0,3.0,0.000,22.000,34.0,30.0,40.0,9.0,9.0,10.0,3.0,3.0,17,79,0.0,0.0,8.0,0.00,0.00,7,24,252,347,609306,29.00,73.00,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,10.0,10.0,-3.0,15.0,15.0,10.0,100.0,11.0,3.0,3.0,4.0,3.0
2,340434758309188355,00,621392873801CH,6213928738,01,CH,2024-08-29 10:48:58.440,2.796,2.60,W,0,0,15.0,473.0,282.0,113.0,2024-08-29 10:09:00,V30,cuboid,G05,V30,cuboid,117365,37,4871,598,3108,112.0,18.0,470.0,319.0,112.0,947.0,223.0,52.0,635.0,1.0,0.0,20.0,2.0,0.000,16.000,8.0,1.0,30.0,13.0,25.0,7.0,7.0,4.0,1,6,-1.0,0.0,4.0,0.00,0.00,0,1,0,7,116518,0.00,0.00,0.0,4.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,3.0,1.0,12.0,11.0,11.0,-1.0,15.0,15.0,9.0,129.0,19.0,3.0,3.0,2.0,2.0
3,340434611735740996,00,62195393230104,6219539323,01,04,2024-08-09 15:03:37.440,1.000,2.00,W,0,0,14.0,304.0,226.0,198.0,2024-08-09 14:43:00,V30,cuboid,G05,V30,cuboid,3861,22,59,187,173,179.0,45.0,320.0,298.0,85.0,983.0,210.0,92.0,471.0,1.0,0.0,18.0,2.0,0.000,3.000,13.0,1.0,30.0,8.0,15.0,7.0,7.0,4.0,0,33,-1.0,0.0,15.0,0.00,1.00,0,0,0,33,1547,1.00,0.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,11.0,9.0,-1.0,15.0,15.0,7.0,100.0,7.0,7.0,7.0,1.0,5.0
4,340433958479425583,00,50180626690107,5018062669,01,07,2024-08-29 08:58:30.200,5.505,5.45,W,0,0,124.0,890.0,567.0,245.0,2024-08-29 08:45:00,V99,cuboid,G10,V99,cuboid,305985,49,7226,1861,6125,342.0,120.0,760.0,833.0,383.0,1863.0,543.0,147.0,1211.0,11.0,0.0,40.0,11.0,0.000,48.000,152.0,98.0,701.0,7.0,7.0,17.0,2.0,2.0,2269,3646,0.0,0.0,-8.0,1.00,1.00,34640,36909,146450,150247,280371,94.00,97.00,0.0,4.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,10.0,5.0,-10.0,15.0,15.0,17.0,100.0,4.0,3.0,3.0,2.0,5.0


Der Import hat  92.43649005889893 Sekunden gedauert!


# Vorbereitung Daten für das Scoring:

In [3]:
GS_validationset.shape

(500000, 90)

In [4]:
# Alle Spaltennamen in Kleinbustaben:
GS_validationset.columns = map(str.lower, GS_validationset.columns)

In [5]:
# Count the number of NaN values
print('Number of NULL-Values in the Dataset: ' + str(GS_validationset.isnull().sum().sum()))

#Spalten mit NULL-Values:
GS_validationset.isnull().sum()

Number of NULL-Values in the Dataset: 234733


sendungs_code                  0
pruefziffer                    0
ekp_no_extented               51
ekp_no                        51
verfahren_nr                   0
teilnahme_nr                   0
event_dt                       0
gewicht_pan                    0
gewicht_real                   0
gewicht_quelle_tmp             0
flag_gewicht_20                0
flag_gewicht_10                0
volumen                        0
laenge                         0
breite                         0
hoehe                          0
erstellugsdatum                0
volumen_grp                    0
parcel_shape                   0
gewicht_pan_grp                0
d_volumen_grp               4427
d_parcel_shape              4427
anzahl                      4427
d_anzahl                    4427
d_anz_diff_pan_weights      4427
d_anz_diff_real_weights     4427
zeitraum_tage               4427
d_hoehe                     4427
d_hoehe_min                 4427
d_hoehe_max                 4427
d_laenge  

### Data Cleaning

In [6]:
#NULL-Werte durch 0 ersetzen:
#change all Type2 NaN values to 'None':
GS_validationset['d_volumen_grp'] = GS_validationset['d_volumen_grp'].fillna(0)
GS_validationset['d_parcel_shape'] = GS_validationset['d_parcel_shape'].fillna(0)
GS_validationset['anzahl'] = GS_validationset['anzahl'].fillna(0)
GS_validationset['zeitraum_tage'] = GS_validationset['zeitraum_tage'].fillna(0)
GS_validationset['d_anzahl'] = GS_validationset['d_anzahl'].fillna(0)
GS_validationset['d_anz_diff_pan_weights'] = GS_validationset['d_anz_diff_pan_weights'].fillna(0)
GS_validationset['d_anz_diff_real_weights'] = GS_validationset['d_anz_diff_real_weights'].fillna(0)
GS_validationset['d_hoehe'] = GS_validationset['d_hoehe'].fillna(0)
GS_validationset['d_hoehe_min'] = GS_validationset['d_hoehe_min'].fillna(0)
GS_validationset['d_hoehe_max'] = GS_validationset['d_hoehe_max'].fillna(0)
GS_validationset['d_breite'] = GS_validationset['d_breite'].fillna(0)
GS_validationset['d_breite_min'] = GS_validationset['d_breite_min'].fillna(0)
GS_validationset['d_breite_max'] = GS_validationset['d_breite_max'].fillna(0)
GS_validationset['d_laenge'] = GS_validationset['d_laenge'].fillna(0)
GS_validationset['d_laenge_min'] = GS_validationset['d_laenge_min'].fillna(0)
GS_validationset['d_laenge_max'] = GS_validationset['d_laenge_max'].fillna(0)
GS_validationset['d_gewicht_real'] = GS_validationset['d_gewicht_real'].fillna(0)
GS_validationset['d_gewicht_real_min'] = GS_validationset['d_gewicht_real_min'].fillna(0)
GS_validationset['d_gewicht_real_max'] = GS_validationset['d_gewicht_real_max'].fillna(0)
GS_validationset['d_gewicht_pan'] = GS_validationset['d_gewicht_pan'].fillna(0)
GS_validationset['d_gewicht_pan_min'] = GS_validationset['d_gewicht_pan_min'].fillna(0)
GS_validationset['d_gewicht_pan_max'] = GS_validationset['d_gewicht_pan_max'].fillna(0)
GS_validationset['d_volumen'] = GS_validationset['d_volumen'].fillna(0)
GS_validationset['d_volumen_min'] = GS_validationset['d_volumen_min'].fillna(0)
GS_validationset['d_volumen_max'] = GS_validationset['d_volumen_max'].fillna(0)
GS_validationset['d_volumengewicht_real'] = GS_validationset['d_volumengewicht_real'].fillna(0)
GS_validationset['d_volumengewicht_pan'] = GS_validationset['d_volumengewicht_pan'].fillna(0)
GS_validationset['d_lbh_summe'] = GS_validationset['d_lbh_summe'].fillna(0)
GS_validationset['d_volumengewicht_pan2'] = GS_validationset['d_volumengewicht_pan2'].fillna(0)
GS_validationset['d_volumengewicht_real2'] = GS_validationset['d_volumengewicht_real2'].fillna(0)
GS_validationset['anzahl_gu_20'] = GS_validationset['anzahl_gu_20'].fillna(0)
GS_validationset['anzahl_gu_10'] = GS_validationset['anzahl_gu_10'].fillna(0)
GS_validationset['d_gewichtsabweichung'] = GS_validationset['d_gewichtsabweichung'].fillna(0)
GS_validationset['d_gewichtsabweichung_min'] = GS_validationset['d_gewichtsabweichung_min'].fillna(0)
GS_validationset['d_gewichtsabweichung_max'] = GS_validationset['d_gewichtsabweichung_max'].fillna(0)
GS_validationset['d_anteil_gu_20'] = GS_validationset['d_anteil_gu_20'].fillna(0)
GS_validationset['d_anteil_gu_10'] = GS_validationset['d_anteil_gu_10'].fillna(0)
GS_validationset['dgewichtsabweichung1'] = GS_validationset['dgewichtsabweichung1'].fillna(0)
GS_validationset['dgewichtsabweichung2'] = GS_validationset['dgewichtsabweichung2'].fillna(0)
GS_validationset['dgewichtsabweichung3'] = GS_validationset['dgewichtsabweichung3'].fillna(0)
GS_validationset['dgewichtsabweichung4'] = GS_validationset['dgewichtsabweichung4'].fillna(0)
GS_validationset['dgewichtsabweichung5'] = GS_validationset['dgewichtsabweichung5'].fillna(0)
GS_validationset['dgewichtsabweichung6'] = GS_validationset['dgewichtsabweichung6'].fillna(0)
GS_validationset['lbh_vh'] = GS_validationset['lbh_vh'].fillna(0)
GS_validationset['diff_kg_sum'] = GS_validationset['diff_kg_sum'].fillna(0)
GS_validationset['anzahl_pan20_korrekt'] = GS_validationset['anzahl_pan20_korrekt'].fillna(0)
GS_validationset['anzahl_pan20'] = GS_validationset['anzahl_pan20'].fillna(0)
GS_validationset['anzahl_pan10_korrekt'] = GS_validationset['anzahl_pan10_korrekt'].fillna(0)
GS_validationset['anzahl_pan10'] = GS_validationset['anzahl_pan10'].fillna(0)
GS_validationset['anzahl_pan_korrekt'] = GS_validationset['anzahl_pan_korrekt'].fillna(0)
GS_validationset['anteil_pan20_korrekt'] = GS_validationset['anteil_pan20_korrekt'].fillna(0)
GS_validationset['anteil_pan10_korrekt'] = GS_validationset['anteil_pan10_korrekt'].fillna(0)
#GS_validationset['pauschalpreis'] = GS_validationset['pauschalpreis'].fillna(0)
GS_validationset['volumengewicht_pan3'] = GS_validationset['volumengewicht_pan3'].fillna(0)


GS_validationset.head()

,sendungs_code,pruefziffer,ekp_no_extented,ekp_no,verfahren_nr,teilnahme_nr,event_dt,gewicht_pan,gewicht_real,gewicht_quelle_tmp,flag_gewicht_20,flag_gewicht_10,volumen,laenge,breite,hoehe,erstellugsdatum,volumen_grp,parcel_shape,gewicht_pan_grp,d_volumen_grp,d_parcel_shape,anzahl,d_anzahl,d_anz_diff_pan_weights,d_anz_diff_real_weights,zeitraum_tage,d_hoehe,d_hoehe_min,d_hoehe_max,d_laenge,d_laenge_min,d_laenge_max,d_breite,d_breite_min,d_breite_max,d_gewicht_real,d_gewicht_real_min,d_gewicht_real_max,d_gewicht_pan,d_gewicht_pan_min,d_gewicht_pan_max,d_volumen,d_volumen_min,d_volumen_max,d_volumengewicht_real,d_volumengewicht_pan,d_lbh_summe,d_volumengewicht_real2,d_volumengewicht_pan2,anzahl_gu_20,anzahl_gu_10,d_gewichtsabweichung,d_gewichtsabweichung_min,d_gewichtsabweichung_max,d_anteil_gu_20,d_anteil_gu_10,anzahl_pan20_korrekt,anzahl_pan20,anzahl_pan10_korrekt,anzahl_pan10,anzahl_pan_korrekt,anteil_pan20_korrekt,anteil_pan10_korrekt,diff_kg_sum,vh_laenge_hoehe,vh_laenge_breite,vh_hoehe_breite,vh_volumen_laenge,vh_volumen_breite,vh_volumen_hoehe,vh_laenge_dlaenge,vh_breite_dbreite,vh_hoehe_dhoehe,vh_volumen_dvolumen,vh_gewicht_dgewicht,vh_gewicht_dgewicht_pan,dgewichtsabweichung1,dgewichtsabweichung2,dgewichtsabweichung3,dgewichtsabweichung4,dgewichtsabweichung5,dgewichtsabweichung6,lbh_summe,lbh_vh,volumengewicht_pan,volumengewicht_pan2,volumengewicht_pan3,vh_volumengewicht_pan,vh_volumengewicht_pan2
0,340434739621298311,00,50266391890104,5026639189,01,04,2024-08-01 11:27:02.120,12.000,12.05,W,0,0,156.0,782.0,605.0,330.0,2024-08-01 11:09:00,V99,cuboid,G15,V99,cuboid,3301.0,10.0,583.0,627.0,310.0,411.0,175.0,630.0,783.0,559.0,1808.0,537.0,221.0,720.0,13.0,1.0,39.0,13.0,1.000,32.000,173.0,98.0,375.0,8.0,8.0,17.0,1.0,1.0,17.0,33.0,0.0,0.0,7.0,1.00,1.00,530.0,547.0,1957.0,1994.0,3001.0,97.00,98.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,10.0,9.0,-10.0,15.0,15.0,17.0,100.0,8.0,1.0,1.0,3.0,3.0
1,340434469949629978,00,62668699240101,6266869924,01,01,2024-08-22 19:37:19.540,1.000,0.35,W,0,0,4.0,153.0,151.0,163.0,2024-08-22 18:15:00,V30,cuboid,G05,V30,cuboid,3302.0,2.0,2.0,89.0,1247.0,158.0,31.0,348.0,253.0,149.0,594.0,221.0,87.0,570.0,1.0,0.0,4.0,1.0,1.000,3.000,9.0,1.0,29.0,11.0,11.0,6.0,6.0,6.0,0.0,0.0,0.0,-1.0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,3100.0,1.00,1.00,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,10.0,10.0,10.0,-1.0,3.0,3.0,5.0,83.0,25.0,5.0,6.0,7.0,3.0
2,340060673132092558,00,500463893801AR,5004638938,01,AR,2024-08-23 12:16:00.580,2.414,2.35,W,0,0,27.0,447.0,308.0,193.0,2024-08-23 11:57:00,V30,cuboid,G05,V30,cuboid,490205.0,167.0,5700.0,877.0,2934.0,148.0,18.0,460.0,375.0,77.0,1180.0,276.0,32.0,588.0,2.0,0.0,35.0,2.0,0.000,25.000,16.0,1.0,30.0,13.0,13.0,8.0,4.0,4.0,65.0,53.0,0.0,0.0,10.0,0.00,0.00,1.0,66.0,4.0,121.0,487849.0,2.00,3.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,10.0,10.0,10.0,-2.0,15.0,15.0,9.0,113.0,9.0,4.0,3.0,2.0,3.0
3,JD000390016886527284,J,63267322590182,6326732259,01,82,2024-08-02 23:59:36.390,0.013,1.30,W,0,0,9.0,310.0,309.0,96.0,2024-08-02 21:54:00,V30,bag,G05,V30,bag,39548.0,102.0,526.0,209.0,385.0,85.0,14.0,314.0,340.0,133.0,642.0,297.0,54.0,468.0,1.0,0.0,30.0,1.0,0.000,5.000,9.0,1.0,30.0,11.0,11.0,7.0,7.0,7.0,1.0,3.0,0.0,0.0,15.0,0.00,0.00,0.0,1.0,0.0,4.0,38066.0,0.00,0.00,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,9.0,10.0,9.0,-1.0,15.0,15.0,7.0,100.0,0.0,15.0,15.0,0.0,6.0
4,35439754719,2,70004224070115,7000422407,01,15,2024-08-09 10:21:02.470,2.000,0.80,W,0,0,3.0,346.0,250.0,40.0,2024-08-09 10:00:00,V30,envelope,G05,V30,envelope,56197.0,2.0,4.0,374.0,21720.0,43.0,1.0,388.0,316.0,26.0,1168.0,236.0,35.0,878.0,1.0,0.0,24.0,2.0,2.000,32.000,3.0,1.0,30.0,33.0,67.0,6.0,6.0,3.0,0.0,20.0,-1.0,-2.0,-8.0,0.00,0.00,1.0,1.0,5.0,25.0,5658.0,100.00,20.00,0.0,9.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,11.0,11.0,10.0,-1.0,15.0,15.0,6.0,100.0,50.0,3.0,3.0,2.0,3.0


In [7]:
# Count the number of NaN values
print('Number of NULL-Values in the Dataset: ' + str(GS_validationset.isnull().sum().sum()))

#Spalten mit NULL-Values:
GS_validationset.isnull().sum()

Number of NULL-Values in the Dataset: 102


sendungs_code                0
pruefziffer                  0
ekp_no_extented             51
ekp_no                      51
verfahren_nr                 0
teilnahme_nr                 0
event_dt                     0
gewicht_pan                  0
gewicht_real                 0
gewicht_quelle_tmp           0
flag_gewicht_20              0
flag_gewicht_10              0
volumen                      0
laenge                       0
breite                       0
hoehe                        0
erstellugsdatum              0
volumen_grp                  0
parcel_shape                 0
gewicht_pan_grp              0
d_volumen_grp                0
d_parcel_shape               0
anzahl                       0
d_anzahl                     0
d_anz_diff_pan_weights       0
d_anz_diff_real_weights      0
zeitraum_tage                0
d_hoehe                      0
d_hoehe_min                  0
d_hoehe_max                  0
d_laenge                     0
d_laenge_min                 0
d_laenge

In [8]:
#Erstelle Backup um das Score-Ergebnis wieder an die orginaldatan anzuspielen:
GS_validationset_Backup = GS_validationset.copy()

In [9]:
# löschen, da Variable bei Prognose nicht verfügbar oder fachliche keinen Sinn macht:
drop_col0 = ['anzahl', 'ekp_no', 'ekp_no_extented', 'erstellugsdatum',  'event_dt',
            'gewicht_quelle', 'sendungs_code', 'teilnahme_nr', 'zeitraum_tage', 'product_code', 'verfahren_nr',
            'pruefziffer', 'anzahl_gu','gewicht_quelle_tmp','d_volumen_grp','d_parcel_shape']
GS_validationset.drop(drop_col0, axis=1, inplace=True, errors='ignore')

In [10]:
# One hot encoding:
 
# Select Features for one hot encoding

DS_one_hot =GS_validationset[['volumen_grp', 'parcel_shape','gewicht_pan_grp']]
    
# Implement one-hot-encoding with get_dummies:

DS_one_hot2 = DS_one_hot.applymap(str)

DS_one_hot3 = pd.get_dummies(DS_one_hot2)

GS_validationset.drop(['volumen_grp', 'parcel_shape','gewicht_pan_grp'], axis=1, inplace=True)

GS_validationset = pd.concat([GS_validationset,DS_one_hot3],axis=1)

GS_validationset.head()

/tmp/ipykernel_124/1933193179.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DS_one_hot2 = DS_one_hot.applymap(str)


,gewicht_pan,gewicht_real,flag_gewicht_20,flag_gewicht_10,volumen,laenge,breite,hoehe,d_anzahl,d_anz_diff_pan_weights,d_anz_diff_real_weights,d_hoehe,d_hoehe_min,d_hoehe_max,d_laenge,d_laenge_min,d_laenge_max,d_breite,d_breite_min,d_breite_max,d_gewicht_real,d_gewicht_real_min,d_gewicht_real_max,d_gewicht_pan,d_gewicht_pan_min,d_gewicht_pan_max,d_volumen,d_volumen_min,d_volumen_max,d_volumengewicht_real,d_volumengewicht_pan,d_lbh_summe,d_volumengewicht_real2,d_volumengewicht_pan2,anzahl_gu_20,anzahl_gu_10,d_gewichtsabweichung,d_gewichtsabweichung_min,d_gewichtsabweichung_max,d_anteil_gu_20,d_anteil_gu_10,anzahl_pan20_korrekt,anzahl_pan20,anzahl_pan10_korrekt,anzahl_pan10,anzahl_pan_korrekt,anteil_pan20_korrekt,anteil_pan10_korrekt,diff_kg_sum,vh_laenge_hoehe,vh_laenge_breite,vh_hoehe_breite,vh_volumen_laenge,vh_volumen_breite,vh_volumen_hoehe,vh_laenge_dlaenge,vh_breite_dbreite,vh_hoehe_dhoehe,vh_volumen_dvolumen,vh_gewicht_dgewicht,vh_gewicht_dgewicht_pan,dgewichtsabweichung1,dgewichtsabweichung2,dgewichtsabweichung3,dgewichtsabweichung4,dgewichtsabweichung5,dgewichtsabweichung6,lbh_summe,lbh_vh,volumengewicht_pan,volumengewicht_pan2,volumengewicht_pan3,vh_volumengewicht_pan,vh_volumengewicht_pan2,volumen_grp_V30,volumen_grp_V40,volumen_grp_V50,volumen_grp_V70,volumen_grp_V90,volumen_grp_V99,parcel_shape_bag,parcel_shape_cuboid,parcel_shape_envelope,parcel_shape_unknown,gewicht_pan_grp_G05,gewicht_pan_grp_G10,gewicht_pan_grp_G15,gewicht_pan_grp_G20,gewicht_pan_grp_G30,gewicht_pan_grp_G99
0,12.000,12.05,0,0,156.0,782.0,605.0,330.0,10.0,583.0,627.0,411.0,175.0,630.0,783.0,559.0,1808.0,537.0,221.0,720.0,13.0,1.0,39.0,13.0,1.000,32.000,173.0,98.0,375.0,8.0,8.0,17.0,1.0,1.0,17.0,33.0,0.0,0.0,7.0,1.00,1.00,530.0,547.0,1957.0,1994.0,3001.0,97.00,98.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,10.0,9.0,-10.0,15.0,15.0,17.0,100.0,8.0,1.0,1.0,3.0,3.0,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False
1,1.000,0.35,0,0,4.0,153.0,151.0,163.0,2.0,2.0,89.0,158.0,31.0,348.0,253.0,149.0,594.0,221.0,87.0,570.0,1.0,0.0,4.0,1.0,1.000,3.000,9.0,1.0,29.0,11.0,11.0,6.0,6.0,6.0,0.0,0.0,0.0,-1.0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,3100.0,1.00,1.00,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,10.0,10.0,10.0,-1.0,3.0,3.0,5.0,83.0,25.0,5.0,6.0,7.0,3.0,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False
2,2.414,2.35,0,0,27.0,447.0,308.0,193.0,167.0,5700.0,877.0,148.0,18.0,460.0,375.0,77.0,1180.0,276.0,32.0,588.0,2.0,0.0,35.0,2.0,0.000,25.000,16.0,1.0,30.0,13.0,13.0,8.0,4.0,4.0,65.0,53.0,0.0,0.0,10.0,0.00,0.00,1.0,66.0,4.0,121.0,487849.0,2.00,3.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,10.0,10.0,10.0,-2.0,15.0,15.0,9.0,113.0,9.0,4.0,3.0,2.0,3.0,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False
3,0.013,1.30,0,0,9.0,310.0,309.0,96.0,102.0,526.0,209.0,85.0,14.0,314.0,340.0,133.0,642.0,297.0,54.0,468.0,1.0,0.0,30.0,1.0,0.000,5.000,9.0,1.0,30.0,11.0,11.0,7.0,7.0,7.0,1.0,3.0,0.0,0.0,15.0,0.00,0.00,0.0,1.0,0.0,4.0,38066.0,0.00,0.00,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,9.0,10.0,9.0,-1.0,15.0,15.0,7.0,100.0,0.0,15.0,15.0,0.0,6.0,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False
4,2.000,0.80,0,0,3.0,346.0,250.0,40.0,2.0,4.0,374.0,43.0,1.0,388.0,316.0,26.0,1168.0,236.0,35.0,878.0,1.0,0.0,24.0,2.0,2.000,32.000,3.0,1.0,30.0,33.0,67.0,6.0,6.0,3.0,0.0,20.0,-1.0,-2.0,-8.0,0.00,0.00,1.0,1.0,5.0,25.0,5658.0,100.00,20.00,0.0,9.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,11.0,11.0,10.0,-1.0,15.0,15.0,6.0,100.0,50.0,3.0,3.0,2.0,3.0,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False


In [11]:
#drop dataframe ds_onehot

In [12]:
# Count the number of NaN values
print('Number of NULL-Values in the Dataset: ' + str(GS_validationset.isnull().sum().sum()))

#Spalten mit NULL-Values:
GS_validationset.isnull().sum()

Number of NULL-Values in the Dataset: 0


gewicht_pan                 0
gewicht_real                0
flag_gewicht_20             0
flag_gewicht_10             0
volumen                     0
laenge                      0
breite                      0
hoehe                       0
d_anzahl                    0
d_anz_diff_pan_weights      0
d_anz_diff_real_weights     0
d_hoehe                     0
d_hoehe_min                 0
d_hoehe_max                 0
d_laenge                    0
d_laenge_min                0
d_laenge_max                0
d_breite                    0
d_breite_min                0
d_breite_max                0
d_gewicht_real              0
d_gewicht_real_min          0
d_gewicht_real_max          0
d_gewicht_pan               0
d_gewicht_pan_min           0
d_gewicht_pan_max           0
d_volumen                   0
d_volumen_min               0
d_volumen_max               0
d_volumengewicht_real       0
d_volumengewicht_pan        0
d_lbh_summe                 0
d_volumengewicht_real2      0
d_volumeng

In [13]:
# dropping target variables 
drop_col = ['gewicht_real',  'flag_gewicht_20', 'flag_gewicht_10']

GS_validationset.drop(drop_col, axis=1, inplace=True, errors='ignore')

print(GS_validationset.shape)

(500000, 87)


In [14]:
GS_validationset.head()

,gewicht_pan,volumen,laenge,breite,hoehe,d_anzahl,d_anz_diff_pan_weights,d_anz_diff_real_weights,d_hoehe,d_hoehe_min,d_hoehe_max,d_laenge,d_laenge_min,d_laenge_max,d_breite,d_breite_min,d_breite_max,d_gewicht_real,d_gewicht_real_min,d_gewicht_real_max,d_gewicht_pan,d_gewicht_pan_min,d_gewicht_pan_max,d_volumen,d_volumen_min,d_volumen_max,d_volumengewicht_real,d_volumengewicht_pan,d_lbh_summe,d_volumengewicht_real2,d_volumengewicht_pan2,anzahl_gu_20,anzahl_gu_10,d_gewichtsabweichung,d_gewichtsabweichung_min,d_gewichtsabweichung_max,d_anteil_gu_20,d_anteil_gu_10,anzahl_pan20_korrekt,anzahl_pan20,anzahl_pan10_korrekt,anzahl_pan10,anzahl_pan_korrekt,anteil_pan20_korrekt,anteil_pan10_korrekt,diff_kg_sum,vh_laenge_hoehe,vh_laenge_breite,vh_hoehe_breite,vh_volumen_laenge,vh_volumen_breite,vh_volumen_hoehe,vh_laenge_dlaenge,vh_breite_dbreite,vh_hoehe_dhoehe,vh_volumen_dvolumen,vh_gewicht_dgewicht,vh_gewicht_dgewicht_pan,dgewichtsabweichung1,dgewichtsabweichung2,dgewichtsabweichung3,dgewichtsabweichung4,dgewichtsabweichung5,dgewichtsabweichung6,lbh_summe,lbh_vh,volumengewicht_pan,volumengewicht_pan2,volumengewicht_pan3,vh_volumengewicht_pan,vh_volumengewicht_pan2,volumen_grp_V30,volumen_grp_V40,volumen_grp_V50,volumen_grp_V70,volumen_grp_V90,volumen_grp_V99,parcel_shape_bag,parcel_shape_cuboid,parcel_shape_envelope,parcel_shape_unknown,gewicht_pan_grp_G05,gewicht_pan_grp_G10,gewicht_pan_grp_G15,gewicht_pan_grp_G20,gewicht_pan_grp_G30,gewicht_pan_grp_G99
0,12.000,156.0,782.0,605.0,330.0,10.0,583.0,627.0,411.0,175.0,630.0,783.0,559.0,1808.0,537.0,221.0,720.0,13.0,1.0,39.0,13.0,1.000,32.000,173.0,98.0,375.0,8.0,8.0,17.0,1.0,1.0,17.0,33.0,0.0,0.0,7.0,1.00,1.00,530.0,547.0,1957.0,1994.0,3001.0,97.00,98.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,10.0,9.0,-10.0,15.0,15.0,17.0,100.0,8.0,1.0,1.0,3.0,3.0,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False
1,1.000,4.0,153.0,151.0,163.0,2.0,2.0,89.0,158.0,31.0,348.0,253.0,149.0,594.0,221.0,87.0,570.0,1.0,0.0,4.0,1.0,1.000,3.000,9.0,1.0,29.0,11.0,11.0,6.0,6.0,6.0,0.0,0.0,0.0,-1.0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,3100.0,1.00,1.00,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,10.0,10.0,10.0,-1.0,3.0,3.0,5.0,83.0,25.0,5.0,6.0,7.0,3.0,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False
2,2.414,27.0,447.0,308.0,193.0,167.0,5700.0,877.0,148.0,18.0,460.0,375.0,77.0,1180.0,276.0,32.0,588.0,2.0,0.0,35.0,2.0,0.000,25.000,16.0,1.0,30.0,13.0,13.0,8.0,4.0,4.0,65.0,53.0,0.0,0.0,10.0,0.00,0.00,1.0,66.0,4.0,121.0,487849.0,2.00,3.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,10.0,10.0,10.0,-2.0,15.0,15.0,9.0,113.0,9.0,4.0,3.0,2.0,3.0,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False
3,0.013,9.0,310.0,309.0,96.0,102.0,526.0,209.0,85.0,14.0,314.0,340.0,133.0,642.0,297.0,54.0,468.0,1.0,0.0,30.0,1.0,0.000,5.000,9.0,1.0,30.0,11.0,11.0,7.0,7.0,7.0,1.0,3.0,0.0,0.0,15.0,0.00,0.00,0.0,1.0,0.0,4.0,38066.0,0.00,0.00,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,9.0,10.0,9.0,-1.0,15.0,15.0,7.0,100.0,0.0,15.0,15.0,0.0,6.0,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False
4,2.000,3.0,346.0,250.0,40.0,2.0,4.0,374.0,43.0,1.0,388.0,316.0,26.0,1168.0,236.0,35.0,878.0,1.0,0.0,24.0,2.0,2.000,32.000,3.0,1.0,30.0,33.0,67.0,6.0,6.0,3.0,0.0,20.0,-1.0,-2.0,-8.0,0.00,0.00,1.0,1.0,5.0,25.0,5658.0,100.00,20.00,0.0,9.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,11.0,11.0,10.0,-1.0,15.0,15.0,6.0,100.0,50.0,3.0,3.0,2.0,3.0,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False


In [15]:
# Model laden                                                                                                                                                                                                          

import pickle
Dateiname = 'GS_RF_Klassifikation_HER_GU_20_nat.pkl'
print(Dateiname)
Speicherort = '/home/jovyan/shared-data/models_herkules/'+ Dateiname                                                                                                                                                                                         

with open(Speicherort, 'rb') as f:
    RF_Klassifikation_GU_20 = pickle.load(f)

GS_RF_Klassifikation_HER_GU_20_nat.pkl


In [16]:
#Prognose erstellen
pred_PGU_20 = RF_Klassifikation_GU_20.predict(GS_validationset)

In [17]:
#Propability der Prognose ermitteln
p_PGU_20= RF_Klassifikation_GU_20.predict_proba(GS_validationset)

In [18]:
# Model laden                                                                                                                                                                                                          

import pickle
Dateiname = 'GS_RF_Prognose_Gewicht_Herkules_nat.pkl'
print(Dateiname)
Speicherort = '/home/jovyan/shared-data/models_herkules/'+ Dateiname                                                                                                                                                                                        

with open(Speicherort, 'rb') as f:
    RF_Prediction_GU_20 = pickle.load(f)

GS_RF_Prognose_Gewicht_Herkules_nat.pkl


In [19]:
pred_weight_20 = RF_Prediction_GU_20.predict(GS_validationset)

In [20]:
#GS_validationset.columns

In [21]:
# # scaling the data
# from sklearn.preprocessing import StandardScaler

# columns_scaled =GS_validationset.columns
# scaler = StandardScaler()

# GS_validationset_scaled = pd.DataFrame(data = GS_validationset)
# GS_validationset_scaled[columns_scaled]=scaler.fit_transform(GS_validationset[columns_scaled])

# # Show an example of a record with scaling applied
# display(GS_validationset_scaled.head(n = 5))

In [22]:
# # Model laden                                                                                                                                                                                                          

# import pickle
# Dateiname = 'GS_SVM_Klassifikation_HER_GU_20_vmadasu.pkl'
# print(Dateiname)
# Speicherort = '/home/jovyan/shared-data/models_herkules/'+ Dateiname                                                                                                                                                                                         

# with open(Speicherort, 'rb') as f:
#     SVM_Klassifikation_GU_20 = pickle.load(f)

In [23]:
# #Prognose erstellen
# pred_PGU_20_SVM = SVM_Klassifikation_GU_20.predict(GS_validationset_scaled)

# #3 min

In [24]:
#Propability der Prognose ermitteln
#p_PGU_20_SVM= SVM_Klassifikation_GU_20.predict_proba(GS_validationset_scaled)

In [25]:
#p_PGU_20_SVM

In [26]:
# Model laden                                                                                                                                                                                                          

#import pickle
#Dateiname = 'GS_SVM_Regression_HER_GU_20_vmadasu.pkl'
#print(Dateiname)
#Speicherort = '/home/jovyan/shared-data/models_herkules/'+ Dateiname                                                                                                                                                                                         

#with open(Speicherort, 'rb') as f:
    #SVM_Regression_GU_20 = pickle.load(f)

In [27]:
#Prognose erstellen
#pred_weight_20_SVM = SVM_Regression_GU_20.predict(GS_validationset)
#2 hours

In [28]:
GS_validationset['pred_PGU_20']=pred_PGU_20

In [29]:
GS_validationset['p_PGU_20']=p_PGU_20[:,1]

In [30]:
GS_validationset['pred_weight_20']=pred_weight_20

In [31]:
#GS_validationset['pred_PGU_20_SVM']=pred_PGU_20_SVM

In [32]:
#GS_validationset['p_PGU_20_SVM']=p_PGU_20_SVM[:,1]

In [33]:
results = GS_validationset[['pred_PGU_20', 'p_PGU_20', 'pred_weight_20']]
results.head()

,pred_PGU_20,p_PGU_20,pred_weight_20
0,0,0.097288,12.940987
1,0,0.000000,0.528086
2,0,0.009892,2.594659
3,0,0.010123,1.428161
4,0,0.000232,0.660655


In [34]:
# Innerjoin:
# Doppler_Scoreergebnis2 = pd.merge(Doppler_Summary_Backup,Doppler_Summary_final[['preds']],left_index = True, right_index = True)

#Leftjoin:
Gewicht_Scoreergebnis = pd.concat([GS_validationset_Backup,results],axis=1)
print(len(Gewicht_Scoreergebnis))

Gewicht_Scoreergebnis.head()


500000


,sendungs_code,pruefziffer,ekp_no_extented,ekp_no,verfahren_nr,teilnahme_nr,event_dt,gewicht_pan,gewicht_real,gewicht_quelle_tmp,flag_gewicht_20,flag_gewicht_10,volumen,laenge,breite,hoehe,erstellugsdatum,volumen_grp,parcel_shape,gewicht_pan_grp,d_volumen_grp,d_parcel_shape,anzahl,d_anzahl,d_anz_diff_pan_weights,d_anz_diff_real_weights,zeitraum_tage,d_hoehe,d_hoehe_min,d_hoehe_max,d_laenge,d_laenge_min,d_laenge_max,d_breite,d_breite_min,d_breite_max,d_gewicht_real,d_gewicht_real_min,d_gewicht_real_max,d_gewicht_pan,d_gewicht_pan_min,d_gewicht_pan_max,d_volumen,d_volumen_min,d_volumen_max,d_volumengewicht_real,d_volumengewicht_pan,d_lbh_summe,d_volumengewicht_real2,d_volumengewicht_pan2,anzahl_gu_20,anzahl_gu_10,d_gewichtsabweichung,d_gewichtsabweichung_min,d_gewichtsabweichung_max,d_anteil_gu_20,d_anteil_gu_10,anzahl_pan20_korrekt,anzahl_pan20,anzahl_pan10_korrekt,anzahl_pan10,anzahl_pan_korrekt,anteil_pan20_korrekt,anteil_pan10_korrekt,diff_kg_sum,vh_laenge_hoehe,vh_laenge_breite,vh_hoehe_breite,vh_volumen_laenge,vh_volumen_breite,vh_volumen_hoehe,vh_laenge_dlaenge,vh_breite_dbreite,vh_hoehe_dhoehe,vh_volumen_dvolumen,vh_gewicht_dgewicht,vh_gewicht_dgewicht_pan,dgewichtsabweichung1,dgewichtsabweichung2,dgewichtsabweichung3,dgewichtsabweichung4,dgewichtsabweichung5,dgewichtsabweichung6,lbh_summe,lbh_vh,volumengewicht_pan,volumengewicht_pan2,volumengewicht_pan3,vh_volumengewicht_pan,vh_volumengewicht_pan2,pred_PGU_20,p_PGU_20,pred_weight_20
0,340434739621298311,00,50266391890104,5026639189,01,04,2024-08-01 11:27:02.120,12.000,12.05,W,0,0,156.0,782.0,605.0,330.0,2024-08-01 11:09:00,V99,cuboid,G15,V99,cuboid,3301.0,10.0,583.0,627.0,310.0,411.0,175.0,630.0,783.0,559.0,1808.0,537.0,221.0,720.0,13.0,1.0,39.0,13.0,1.000,32.000,173.0,98.0,375.0,8.0,8.0,17.0,1.0,1.0,17.0,33.0,0.0,0.0,7.0,1.00,1.00,530.0,547.0,1957.0,1994.0,3001.0,97.00,98.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,10.0,9.0,-10.0,15.0,15.0,17.0,100.0,8.0,1.0,1.0,3.0,3.0,0,0.097288,12.940987
1,340434469949629978,00,62668699240101,6266869924,01,01,2024-08-22 19:37:19.540,1.000,0.35,W,0,0,4.0,153.0,151.0,163.0,2024-08-22 18:15:00,V30,cuboid,G05,V30,cuboid,3302.0,2.0,2.0,89.0,1247.0,158.0,31.0,348.0,253.0,149.0,594.0,221.0,87.0,570.0,1.0,0.0,4.0,1.0,1.000,3.000,9.0,1.0,29.0,11.0,11.0,6.0,6.0,6.0,0.0,0.0,0.0,-1.0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,3100.0,1.00,1.00,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,10.0,10.0,10.0,-1.0,3.0,3.0,5.0,83.0,25.0,5.0,6.0,7.0,3.0,0,0.000000,0.528086
2,340060673132092558,00,500463893801AR,5004638938,01,AR,2024-08-23 12:16:00.580,2.414,2.35,W,0,0,27.0,447.0,308.0,193.0,2024-08-23 11:57:00,V30,cuboid,G05,V30,cuboid,490205.0,167.0,5700.0,877.0,2934.0,148.0,18.0,460.0,375.0,77.0,1180.0,276.0,32.0,588.0,2.0,0.0,35.0,2.0,0.000,25.000,16.0,1.0,30.0,13.0,13.0,8.0,4.0,4.0,65.0,53.0,0.0,0.0,10.0,0.00,0.00,1.0,66.0,4.0,121.0,487849.0,2.00,3.00,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,10.0,10.0,10.0,-2.0,15.0,15.0,9.0,113.0,9.0,4.0,3.0,2.0,3.0,0,0.009892,2.594659
3,JD000390016886527284,J,63267322590182,6326732259,01,82,2024-08-02 23:59:36.390,0.013,1.30,W,0,0,9.0,310.0,309.0,96.0,2024-08-02 21:54:00,V30,bag,G05,V30,bag,39548.0,102.0,526.0,209.0,385.0,85.0,14.0,314.0,340.0,133.0,642.0,297.0,54.0,468.0,1.0,0.0,30.0,1.0,0.000,5.000,9.0,1.0,30.0,11.0,11.0,7.0,7.0,7.0,1.0,3.0,0.0,0.0,15.0,0.00,0.00,0.0,1.0,0.0,4.0,38066.0,0.00,0.00,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,9.0,10.0,9.0,-1.0,15.0,15.0,7.0,100.0,0.0,15.0,15.0,0.0,6.0,0,0.010123,1.428161
4,35439754719,2,70004224070115,7000422407,01,15,2024-08-09 10:21:02.470,2.000,0.80,W,0,0,3.0,346.0,250.0,40.0,2024-08-09 10:00:00,V30,envelope,G05,V30,envelope,56197.0,2.0,4.0,374.0,21720.0,43.0,1.0,388.0,316.0,26.0,1168.0,236.0,35.0,878.0,1.0,0.0,24.0,2.0,2.000,32.000,3.0,1.0,30.0,33.0,67.0,6.0,6.0,3.0,0.0,20.0,-1.0,-2.0,-8.0,0.00,0.00,1.0,1.0,5.0,25.0,5658.0,100.00,20.00,0.0,9.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,11.0,11.0,10.0,-1.0,15.0,15.0,6.0,100.0,50.0,3.0

In [35]:
Gewicht_Scoreergebnis.shape

(500000, 93)

In [36]:
Gewicht_Scoreergebnis = Gewicht_Scoreergebnis.astype({"gewicht_real": float, "gewicht_pan": float,
                                                      "pred_weight_20": float,
                                                      "pred_PGU_20": float, "p_PGU_20": float,
                                                     
                                                      'sendungs_code' : str, 	'ekp_no_extented'  : str,	'ekp_no'  : str,
                                                      'verfahren_nr'  : str,	'teilnahme_nr'  : str,	'event_dt'  : str,	'erstellugsdatum'  : str,	
                                                       })

drop_col = ['volumen',	'laenge',	'breite',	'hoehe',	'd_anzahl',	'd_hoehe',	'd_laenge',	'd_breite',	'd_gewicht_real',	'd_gewicht_pan',	'd_volumengewicht_real',
           'd_volumengewicht_pan',	'd_volumen',	'd_volumengewicht_real2',	'd_volumengewicht_pan2',	'd_anzahl_nat',	'd_hoehe_nat',	'd_laenge_nat',	'd_breite_nat',	
           'd_gewicht_pan_nat',	'd_volumen_nat',	'd_volumengewicht_pan_nat',	'd_volumengewicht_pan2_nat',	'vh_laenge_hoehe',	'vh_laenge_breite',	'vh_hoehe_breite',
           'vh_laenge_dlaenge',	'vh_laenge_dlaenge_nat',	'vh_breite_dbreite',	'vh_breite_dbreite_nat',	'vh_hoehe_dhoehe',	'vh_hoehe_dhoehe_nat',	'vh_volumen_dvolumen',
           'vh_volumen_dvolumen_nat',	'vh_gewicht_dgewicht',	'dgewichtsabweichung1',	'dgewichtsabweichung2',	'dgewichtsabweichung3',
            'gewicht_quelle_tmp', 'anzahl', 'zeitraum_tage',
            'anzahl_gu', 'd_gewichtsabweichung', 'd_gewichtsabweichung_min', 'd_gewichtsabweichung_max','d_anteil_gu', 
           'volumengewicht_pan', 'vh_gewicht_dgewicht_pan', 'lbh_summe', 'lbh_vh', 
           'volumengewicht_pan2', 'vh_volumengewicht_pan', 'vh_volumengewicht_pan2', 
           'd_lbh_summe', 'lbh_summe','volumen_grp','parcel_shape', 'd_volumen_grp','d_parcel_shape', 'd_anz_diff_pan_weights',
           'd_anz_diff_real_weights','d_hoehe_min','d_hoehe_max', 'd_laenge_min', 'd_laenge_max', 'd_breite_min', 'd_breite_max', 
            'd_gewicht_real_min', 'd_gewicht_real_max', 'd_gewicht_pan_min', 'd_gewicht_pan_max', 'd_volumen_min', 'd_volumen_max', 
            'anzahl_gu_20', 'anzahl_gu_10', 'd_anteil_gu_20', 'd_anteil_gu_10', 'gewicht_quelle', 'dgewichtsabweichung4', 
            'dgewichtsabweichung5','pauschalpreis','gewicht_pan_grp','anzahl_pan20_korrekt',	'anzahl_pan20','anzahl_pan10_korrekt',
            'anzahl_pan10','anzahl_pan_korrekt',	'anteil_pan20_korrekt',	'anteil_pan10_korrekt','diff_kg_sum','vh_volumen_laenge','vh_volumen_breite', 'vh_volumen_hoehe','dgewichtsabweichung6','volumengewicht_pan3']

Gewicht_Scoreergebnis.drop(drop_col, inplace=True, axis=1, errors='ignore')
print(len(Gewicht_Scoreergebnis))
Gewicht_Scoreergebnis.head()


500000


,sendungs_code,pruefziffer,ekp_no_extented,ekp_no,verfahren_nr,teilnahme_nr,event_dt,gewicht_pan,gewicht_real,flag_gewicht_20,flag_gewicht_10,erstellugsdatum,pred_PGU_20,p_PGU_20,pred_weight_20
0,340434739621298311,00,50266391890104,5026639189,01,04,2024-08-01 11:27:02.120,12.000,12.05,0,0,2024-08-01 11:09:00,0.0,0.097288,12.940987
1,340434469949629978,00,62668699240101,6266869924,01,01,2024-08-22 19:37:19.540,1.000,0.35,0,0,2024-08-22 18:15:00,0.0,0.000000,0.528086
2,340060673132092558,00,500463893801AR,5004638938,01,AR,2024-08-23 12:16:00.580,2.414,2.35,0,0,2024-08-23 11:57:00,0.0,0.009892,2.594659
3,JD000390016886527284,J,63267322590182,6326732259,01,82,2024-08-02 23:59:36.390,0.013,1.30,0,0,2024-08-02 21:54:00,0.0,0.010123,1.428161
4,35439754719,2,70004224070115,7000422407,01,15,2024-08-09 10:21:02.470,2.000,0.80,0,0,2024-08-09 10:00:00,0.0,0.000232,0.660655


In [37]:
Gewicht_Scoreergebnis.shape

(500000, 15)

# Upload der Daten auf die Teradata:

In [38]:
Zieltabelle = "Herkules_nat_Scorerg_valid_1"
Index = 'sendungs_code'
 
# Falls notwendig löschen der neuen Tabelle
drop_table(f'''DBX_DWH_SBX_GB23_PRD.{Zieltabelle}''')

# Erstellen der neuen Tabelle    
td_connection.execute_sql(f'''
    CREATE MULTISET TABLE DBX_DWH_SBX_GB23_PRD.{Zieltabelle}(
        sendungs_code         varchar(42),
        pruefziffer           varchar(5),
        ekp_no_extented       varchar(20),
        ekp_no                varchar(10),
        verfahren_nr          varchar(2),
        teilnahme_nr          varchar(2),
        event_dt              varchar(25),
        gewicht_pan           float,
        gewicht_real          float,
        flag_gewicht_20       integer,        
        flag_gewicht_10       integer,
        erstellugsdatum       varchar(25),
        pred_PGU_20           integer,
        p_PGU_20               float,
        pred_weight_20         float
    
     
    )PRIMARY INDEX({Index})''')
 
print(f'''DBX_DWH_SBX_GB23_PRD.{Zieltabelle} initialised with PI {Index}.''')


# Upload
 
td_connection._upload_df(schema='DBX_DWH_SBX_GB23_PRD', table_name=Zieltabelle, df=Gewicht_Scoreergebnis)
print(f'''Data uploaded into table {Zieltabelle}.''')
display(td_connection.select(f'''select count(1) from DBX_DWH_SBX_GB23_PRD.{Zieltabelle}'''))

DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_1 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_1 initialised with PI sendungs_code.
Data uploaded into table Herkules_nat_Scorerg_valid_1.


,Count(1)
0,500000


# Vorbereitung 

In [39]:
display(td_connection.select("select Flag_Gewicht_20, count(1) as anzahl from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_1 group by 1 order by 1"))

,flag_gewicht_20,anzahl
0,0,498430
1,1,1570


In [40]:
# Relevanz auf AbrNR-Ebene ermitteln:
 
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_Modelrelevanz_abrnr'
SQL = f'''
 
 
           SELECT 	EKP_NO_Extented as AbrNR, 
                    case when sum(pred_PGU_20*1.00) = 0 then 0 else sum(pred_PGU_20*flag_gewicht_20*1.0000) / sum(pred_PGU_20*1.00) end as Relevanz_pred_PGU, 
                    sum(pred_PGU_20) as Anzahl
           FROM DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_1
           group by AbrNR
 
 
    '''

Index = 'AbrNR'
create_table(Tabelle, SQL, Index)

DBX_DWH_SBX_GB23_PRD.Herkules_nat_Modelrelevanz_abrnr dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_nat_Modelrelevanz_abrnr initialised with PI AbrNR
31560 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_nat_Modelrelevanz_abrnr.
 


In [41]:
# Relevanz auf AbrNR-Ebene anspielen:
 
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_2'
SQL = f'''
 
    Select A.*, B.Relevanz_pred_PGU, B.Anzahl
    from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_1 as A
    left join DBX_DWH_SBX_GB23_PRD.Herkules_nat_Modelrelevanz_abrnr as B
    on A.EKP_NO_Extented = B.AbrNR
 
    '''
Index = 'Sendungs_code'
create_table(Tabelle, SQL, Index)
# display(td_connection.select(f'''select * from {Tabelle} order by Anzahl desc '''))

DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_2 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_2 initialised with PI Sendungs_code
500000 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_2.
 


In [42]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3'
SQL = f'''
   SELECT 	a.*, 
   

           case   
                    when (p_PGU_20 >= 0.80        and (Relevanz_pred_PGU > 0.3 )    )     then 1
                    when (p_PGU_20 >= 0.70        and (Relevanz_pred_PGU > 0.4 )    )     then 1 
                    when (p_PGU_20 >= 0.60        and (Relevanz_pred_PGU > 0.5 )     )     then 1 
                    when (p_PGU_20 >= 0.50        and (Relevanz_pred_PGU > 0.6 )     )     then 1   
                    when (p_PGU_20 >= 0.40        and (Relevanz_pred_PGU > 0.70)  and   pred_weight_20 >21  )     then 1
                    when (p_PGU_20 >= 0.30        and (Relevanz_pred_PGU > 0.75)  and   pred_weight_20 >21    )     then 1
                    when (p_PGU_20 >= 0.25        and Relevanz_pred_PGU > 0.80    and   pred_weight_20 >21   )     then 1   
                    when (p_PGU_20 >= 0.10        and Relevanz_pred_PGU > 0.90    and   pred_weight_20 >21 )     then 1 
                    else 0
            end as Flag_weight_model_20,
            
            
            Flag_Gewicht_20 as FLAG_weight_greater_real_20
            
   FROM DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_2 as a

    '''
Index = 'Sendungs_code'
create_table(Tabelle, SQL, Index)


#RF results
display(td_connection.select(f'''SELECT 
    (SUM(CASE WHEN FLAG_weight_greater_real_20 = pred_PGU_20 THEN 1 ELSE 0 END) * 1.0000 / COUNT(sendungs_code)) AS accuracy from {Tabelle} '''))

display(td_connection.select(f'''select sum(pred_PGU_20*FLAG_weight_greater_real_20*1.0000) / sum(pred_PGU_20*1.00) as Precision_PGU_model from {Tabelle} '''))
display(td_connection.select(f'''select sum(pred_PGU_20*FLAG_weight_greater_real_20*1.0000) / sum(FLAG_weight_greater_real_20*1.00) as Recall_PGU_model from {Tabelle} '''))

display(td_connection.select(f'''select pred_PGU_20, FLAG_weight_greater_real_20, count(sendungs_code) as Anzahl from {Tabelle} group by 1, 2 order by 1, 2'''))


#SVM results
#display(td_connection.select(f'''select sum(pred_PGU_20_SVM*FLAG_weight_greater_real_20*1.0000) / sum(pred_PGU_20_SVM*1.00) as Relevanz_PGU_SVM_model from {Tabelle} '''))
#display(td_connection.select(f'''select sum(pred_PGU_20_SVM*FLAG_weight_greater_real_20*1.0000) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_PGU_SVM_model from {Tabelle} '''))
#display(td_connection.select(f'''select pred_PGU_20_SVM, FLAG_weight_greater_real_20, count(sendungs_code) as Anzahl from {Tabelle} group by 1, 2 order by 1, 2'''))



#combined model results
display(td_connection.select(f'''select  sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.0000) / sum(Flag_weight_model_20*1.00) as Relevanz_combined_models from {Tabelle} '''))
display(td_connection.select(f'''select  sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.0000) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_combined_model from {Tabelle} '''))
display(td_connection.select(f'''select Flag_weight_model_20, FLAG_weight_greater_real_20, count(sendungs_code) as Anzahl from {Tabelle} group by 1, 2 order by 1, 2'''))



DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3 initialised with PI Sendungs_code
500000 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3.
 


,accuracy
0,0.9889


,Precision_PGU_model
0,0.1968


,Recall_PGU_model
0,0.8242


,pred_PGU_20,FLAG_weight_greater_real_20,Anzahl
0,0,0,493148
1,0,1,276
2,1,0,5282
3,1,1,1294


,Relevanz_combined_models
0,0.6771


,Trefferquote_combined_model
0,0.5771


,Flag_weight_model_20,FLAG_weight_greater_real_20,Anzahl
0,0,0,497998
1,0,1,664
2,1,0,432
3,1,1,906


In [43]:
display(td_connection.select(f'''select * from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3  where pred_PGU_20 <> FLAG_weight_greater_real_20 '''))

display(td_connection.select(f'''select count(sendungs_code) as cust, ekp_no, avg(gewicht_real-gewicht_pan) as diff_kg from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3  where pred_PGU_20 <> FLAG_weight_greater_real_20  group by 2 order by 1 desc'''))

#display(td_connection.select(f'''select  * from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3  where pred_PGU_20 <> FLAG_weight_greater_real_20  and ekp_no ='5018062669' '''))

#display(td_connection.select(f'''select  * from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3  where pred_PGU_20 <> FLAG_weight_greater_real_20  and ekp_no ='5012308772' order by 8 '''))

#display(td_connection.select(f'''select  * from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3  where pred_PGU_20 <> FLAG_weight_greater_real_20  and ekp_no ='6294190264' '''))

#display(td_connection.select(f'''select  * from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3  where pred_PGU_20 <> FLAG_weight_greater_real_20  and ekp_no ='6196734418' '''))

#display(td_connection.select(f'''select  * from DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3  where pred_PGU_20 <> FLAG_weight_greater_real_20  and ekp_no ='6063558858' '''))


#5018062669
#5012308772
#6294190264
#6196734418
#6063558858

,sendungs_code,pruefziffer,ekp_no_extented,ekp_no,verfahren_nr,teilnahme_nr,event_dt,gewicht_pan,gewicht_real,flag_gewicht_20,flag_gewicht_10,erstellugsdatum,pred_PGU_20,p_PGU_20,pred_weight_20,Relevanz_pred_PGU,Anzahl,Flag_weight_model_20,FLAG_weight_greater_real_20
0,340434733908544771,00,50123087720148,5012308772,01,48,2024-08-02 17:14:20.600,12.242,14.90,0,0,2024-08-02 17:00:00,1,0.692609,19.571039,0.1470,313,0,0
1,340434762108197432,00,63662364950101,6366236495,01,01,2024-08-20 21:13:51.180,2.979,4.60,0,0,2024-08-20 19:34:00,1,0.529915,16.230611,0.1429,14,0,0
2,46628152800,7,50112187680102,5011218768,01,02,2024-08-28 18:03:59.580,1.000,12.25,0,1,2024-08-28 17:45:00,1,0.956728,24.844048,0.2500,4,0,0
3,340433988141984704,00,50659895920103,5065989592,01,03,2024-08-19 19:49:31.700,5.000,7.85,0,0,2024-08-19 18:12:00,1,0.912941,20.271735,0.0000,2,0,0
4,44566169351,5,50551523420112,5055152342,01,12,2024-08-21 12:14:25.220,5.000,18.60,0,1,2024-08-21 11:45:00,1,0.972080,21.016950,0.1429,42,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5553,340433958475687541,00,50180626690107,5018062669,01,07,2024-08-15 15:26:32.150,18.435,18.40,0,0,2024-08-15 15:01:00,1,0.547979,19.694176,0.0583,120,0,0
5554,340433958473951248,00,50180626690107,5018062669,01,07,2024-08-08 15:38:59.750,18.420,18.45,0,0,2024-08-08 15:05:00,1,0.549343,19.642766,0.0583,120,0,0
5555,340434262760855829,00,61062835850101,6106283585,01,01,2024-08-13 19:08:34.100,9.200,9.10,0,0,2024-08-13 18:00:00,1,0.597971,17.980419,0.0000,2,0,0
5556,340434623822957088,00,63236371690105,6323637169,01,05,2024-08-14 10:36:20.940,18.000,2.65,0,0,2024-08-14 10:04:00,1,0.600968,19.451154,0.2500,4,0,0


,cust,ekp_no,diff_kg
0,322,5012308772,1.212217
1,141,6295644417,0.543511
2,127,5018062669,1.410189
3,104,6063558858,0.556250
4,75,6329242360,0.026933
...,...,...,...
1759,1,6068786791,8.500000
1760,1,6285854868,-0.900000
1761,1,5025053441,4.810000
1762,1,5017200509,2.500000


In [ ]:
now = date.today()
day_akt = now.day
jahr_akt = now.year
monat_akt = now.month 

Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_Report_Modelguete_tmp'

SQL = f'''

   SELECT  'Herkules_national_GK_20' as Algorithm_name,
   
   
           {jahr_akt} as Jahr,

           {monat_akt} as Monat, 
           
           {day_akt} as Day_of_month,
        

           sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.00) / sum(Flag_weight_model_20*1.00) as Relevanz_combined_models,

           sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.00) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_combined_model,

           

           sum(pred_PGU_20*FLAG_weight_greater_real_20*1.00) / sum(pred_PGU_20*1.00) as Relevanz_PGU_model,

           sum(pred_PGU_20*FLAG_weight_greater_real_20*1.00) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_PGU_model

           /*sum(pred_PGU_20_SVM*FLAG_weight_greater_real_20*1.00) / sum(pred_PGU_20_SVM*1.00) as Relevanz_PGU_SVM_model,

           sum(pred_PGU_20_SVM*FLAG_weight_greater_real_20*1.00) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_PGU_SVM_model*/


   FROM DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_valid_3 AS a

 
    '''

Index = 'Jahr, Monat,Day_of_month'

create_table(Tabelle, SQL, Index)

display(td_connection.select(f'''select * from {Tabelle} '''))

In [ ]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_Report_Modelguete_tmp2'

SQL = f'''

   SELECT  *

   FROM DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_Report_Modelguete_tmp 
   
   
   union all 
   
   SELECT  *
   
   FROM DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_Report_Modelguete 

 
    '''

Index = 'Jahr, Monat,Day_of_month'

create_table(Tabelle, SQL, Index)

display(td_connection.select(f'''select * from {Tabelle} '''))

In [ ]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_Report_Modelguete'

SQL = f'''

   SELECT  *
           
   FROM DBX_DWH_SBX_GB23_PRD.Herkules_nat_Scorerg_Report_Modelguete_tmp2
   
   
 
    '''

Index = 'Jahr, Monat,Day_of_month'

create_table(Tabelle, SQL, Index)

display(td_connection.select(f'''select * from {Tabelle} '''))

In [ ]:
startzeit = time.time()

Used_Sandbox = "DBX_DWH_SBX_GB23_PRD"  
Tabelle = "Herkules_nat_Scorerg_Report_Modelguete"
target_schema = 'DBX_DWH_SBX_GB23_PRD'

data =td_connection.select(f''' SELECT  * FROM {Used_Sandbox}.{Tabelle}  ''')
data.head()

endzeit = time.time()-startzeit

print("Der Import hat ", endzeit , "Sekunden gedauert!")


In [180]:
import pandas as pd

# Create a DataFrame with data
df = pd.DataFrame(data)

# Write DataFrame to an Excel file

Dateiname = 'Results_Daten_Download.xlsx'
Speicherort = '/home/jovyan/prd-sbx-18b4-mp-pvc-esi/04_Gewicht/Analysen/' +Dateiname
df.to_excel(Speicherort, index=False)


In [ ]:
#340434166011458497 difference between pan weight and real weight is more than 20kg
#340434468150355324 pan weight and real weights are similar then why model predicted wrong?


#5018062669, 5012308772, 6294190264, 6196734418, 6063558858  this customers had highest number of errors 
